# Proyecto de detección de fraudes de transacciones con Tarjetas de Crédito (TDC)

## Definición de problema y objetivo

About the Dataset

This is a simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 1000 customers doing transactions with a pool of 800 merchants.

### Descripción de los datos

### Métricas de evaluación

## Importando librerías

In [11]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split


## Carga de datos

In [3]:
test_original = pd.read_csv('C:/Users/ramos/Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTest.csv')
train_original = pd.read_csv('C:/Users/ramos/Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTrain.csv')

# test_original = pd.read_csv('C:/Users/omarh\Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTest.csv')
# train_original = pd.read_csv('C:/Users/omarh\Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTrain.csv')


### Conociendo el contenido de los dataframes:

In [4]:
# test_original.info()
# train_original.info()

In [5]:
#De manera intuitiva podemos saber que los dataframes tienen las mismas columnas, sin embargo realizamos una verificación
# Vamos a unificar los datos para realizar el EDA general, posteriormente vamos separar los datos de entrenamiento y prueba

raw_data = pd.concat([test_original,train_original], axis=0)
#eliminamos la primer columna "Unnamed:0" que es un duplicado del índice
raw_data = raw_data.drop(columns=raw_data.columns[0],axis=1)

raw_data.head(3)
raw_data.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1852394 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop               

#### **Descripción del Dataframe**
tenemos un dataframe con 1,852,394 registros no nulos

##### Descripción de columnas
* trans_date_trans_time	 : Fecha de transacción 
    * tipo object, *_**será cambiado a datetime**_*
* ccnum : Número de transacción 
    * tipo int64
* merchant : nombe del comercio donde se hizo la transacción
    * tipo object
* category : categoría
    * tipo object
* amt : monto de la transacción
    * tipo float64
* first : primer nombre del titular de la tarjeta
    * tipo object
* last : apellido del titular de la tarjeta
     * tipo object
* gender: género del titular de la tarjeta
    * tipo object
* street : calle del titular
    * tipo object
* city : ciudad del titular
    * tipo object
* state :  estado del titular
    * tipo object
* zip : código postal del titular
    * tipo int64
* lat : latitud asociada a la dirección del titular
    * tipo float64 
* long : longitud asociada a la dirección del titular
    * tipo float64
* city_pop : población de la ciudad del titular de la tarjeta
    * tipo int64
* job : profesión del titular
    * tipo object
* dob : fecha de nacimiento del titular
    * tipo object, *_**será cambiado a datetime**_*
* trans_num : número único de transacción
    * tipo object
* unix_time : tiempo de la transacción en formato unix
    * tipo int64 , *_**Será eliminado**_*
* merch_lat : latitud de la ubicación del comerciante
    * tipo float64
* merch_long : longitud de la ubicación del comerciante
    * tipo float64
* is_fraud : es fraude 
    * tipo int64

## Preprocesamiento de datos

### Trabajando con dataset **raw_data**
* Eliminación de columnas innecesarias
* Conversión de tipo de datos en columnas
* Verificación de nulos
* Eliminación de duplicados
* Creación de dataframe con una muestra estratificada

In [10]:
raw_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [8]:
#Eliminando columna 'unix_time'
raw_data = raw_data.drop(columns='unix_time')


In [82]:
#convirtiendo 'trans_date_trans_time'  a datetime
raw_data['trans_date_trans_time'] = pd.to_datetime(raw_data['trans_date_trans_time'])
display(raw_data['trans_date_trans_time'].sample(2))

raw_data['dob'] = pd.to_datetime(raw_data['dob'])
display(raw_data['dob'].sample(2))


825234   2019-12-09 15:22:40
965719   2020-01-25 09:08:44
Name: trans_date_trans_time, dtype: datetime64[ns]

582678   2000-02-20
96910    1988-09-15
Name: dob, dtype: datetime64[ns]

In [83]:
#Verificación de nulos
raw_data.isnull().any()


trans_date_trans_time    False
cc_num                   False
merchant                 False
category                 False
amt                      False
first                    False
last                     False
gender                   False
street                   False
city                     False
state                    False
zip                      False
lat                      False
long                     False
city_pop                 False
job                      False
dob                      False
trans_num                False
unix_time                False
merch_lat                False
merch_long               False
is_fraud                 False
dtype: bool

In [12]:
#Eliminación de duplicados
print('filas antes de eliminación de duplicados:',raw_data.shape[0] )
raw2=raw_data.drop_duplicates()
print('Filas después de eliminación de duplicados',raw2.shape[0])
raw_data.shape

filas antes de eliminación de duplicados: 1852394
Filas después de eliminación de duplicados 1852394


(1852394, 21)

In [15]:
#Creando una copia del dataframe para trabajar
# data = raw_data.copy()

In [17]:
#Creando un dataframe con una muestra estratificada del 50%
sampled_data,_ = train_test_split(raw_data,
                                  test_size=0.5,
                                    stratify=raw_data['is_fraud'],
                                      random_state=1234)

In [ ]:
sampled_data.shape

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,zip,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud
628645,2019-09-23 22:56:10,30135235368675,fraud_Lynch Ltd,shopping_pos,10.87,Amanda,Williams,F,02018 Gary Key Apt. 911,Independence,...,64058,39.1412,-94.3515,123373,"Engineer, production",1992-11-27,9ddccdbef4e9260f9eae1bb7613b2047,38.588874,-93.665390,0
834727,2019-12-12 04:43:39,6011860238257910,"fraud_Greenholt, Jacobi and Gleason",gas_transport,43.92,Lisa,Mendez,F,44259 Beth Station Suite 215,Lahoma,...,73754,36.3850,-98.0727,1078,"Programme researcher, broadcasting/film/video",1952-07-06,4f09623ca35aaf3fdaf69c0352f1ee60,36.493311,-98.993213,0


## Análisis exploratorio de los datos (EDA)

### Unificando y analizando dataframes

### Conclusiones EDA

## Creando dataset para el modelo

### Imputación de nulos

## Análisis y Selección de características (Feature engineering)

## Codificación y estandarización de los datos

### Categorización de datos 
(Onehot por ejemplo)
evaluar mantener una copia de datos sin escalar

### Escalando datos

## Manejo de desabalanceo de datos

## Creación del modelo y modelo benchmark